# <center> Unit-Value Conversion

In [76]:
import pandas as pd
import json
import os
import numpy as np

In [77]:
country_codes = pd.read_csv('../../data/raw_trade_data/BACI_country_codes.csv')
country_codes

,country_code,country_name,country_iso2,country_iso3
0,4,Afghanistan,AF,AFG
1,8,Albania,AL,ALB
2,12,Algeria,DZ,DZA
3,16,American Samoa,AS,ASM
4,20,Andorra,AD,AND
...,...,...,...,...
233,876,Wallis and Futuna Isds,WF,WLF
234,882,Samoa,WS,WSM
235,887,Yemen,YE,YEM
236,891,Serbia and Montenegro (...2005),CS,SCG


In [78]:
total_trade_volume = pd.read_csv('../../data/raw_trade_data/BACI_total_trade_volume_2023.csv')

# IMPORTANT: Convert v from thousands of USD to USD
total_trade_volume['v'] = total_trade_volume['v'] * 1000

total_trade_volume

,t,i,j,k,v,q
0,2023,4,20,200290,8013.0,5.830
1,2023,4,31,252620,9808.0,77.150
2,2023,4,31,680221,33788.0,76.655
3,2023,4,31,761510,6604.0,2.368
4,2023,4,31,860900,1500.0,2.200
...,...,...,...,...,...,...
11232734,2023,894,854,903180,8144.0,0.101
11232735,2023,894,858,240120,619897.0,99.000
11232736,2023,894,858,630900,981.0,0.446
11232737,2023,894,858,820840,4743.0,0.191


In [79]:
# Create a copy to avoid modifying the original
df = total_trade_volume.copy()

# Extract HS 2-digit chapter from k
# For 5-digit codes (70310), take the first digit
# For 6-digit codes, take the first two digits
df['hs_chapter'] = df['k'].astype(str).apply(
    lambda x: int(x[0]) if len(x) == 5 else int(x[:2])
)

# Remove rows where weight (q) is NaN for conversion factor calculations
df_clean = df.dropna(subset=['q'])

# Merge with country codes to get country names for exporters and importers
df_clean = df_clean.merge(
    country_codes[['country_code', 'country_name']], 
    left_on='i', 
    right_on='country_code', 
    how='left'
).rename(columns={'country_name': 'exporter_name'}).drop(columns=['country_code'])

df_clean = df_clean.merge(
    country_codes[['country_code', 'country_name']], 
    left_on='j', 
    right_on='country_code', 
    how='left'
).rename(columns={'country_name': 'importer_name'}).drop(columns=['country_code'])

print("="*60)
print("PART 1: CREATING GENERAL CONVERSION FACTOR JSON FILE")
print("="*60)

# Create output directory if it doesn't exist
output_dir = 'trade_volume_conversion_output'
os.makedirs(output_dir, exist_ok=True)

# ===== GENERAL JSON: Average by HS chapter only =====
# Group by HS chapter across all exporter-importer pairs
general_grouped = df_clean.groupby('hs_chapter').agg({
    'v': 'sum',  # total value in USD across all trade flows
    'q': 'sum'   # total weight in metric tons across all trade flows
}).reset_index()

# Calculate the average conversion factor for each chapter
general_grouped['conversion_factor'] = general_grouped['v'] / general_grouped['q']

# Remove any rows where q was 0
general_grouped = general_grouped[general_grouped['q'] > 0]

# Convert to dictionary for JSON export
general_conversion_dict = {}
for _, row in general_grouped.iterrows():
    chapter = int(row['hs_chapter'])
    general_conversion_dict[chapter] = {
        'conversion_factor': float(row['conversion_factor']),
        'total_value_usd': float(row['v']),
        'total_weight_tons': float(row['q'])
    }

# Export general JSON
general_output_path = os.path.join(output_dir, 'trade_volume_conversion_general.json')
with open(general_output_path, 'w') as f:
    json.dump(general_conversion_dict, f, indent=2)

print(f"✓ General conversion factors exported to: {general_output_path}")
print(f"  Number of HS chapters: {len(general_grouped)}")

print("\n" + "="*60)
print("PART 2: CREATING TRADE MATRICES")
print("="*60)

# Get all unique countries (from both exporters and importers)
all_country_ids = pd.concat([df['i'], df['j']]).unique()
all_country_ids = sorted(all_country_ids)

# Create a mapping from country_code to country_name
country_map = dict(zip(country_codes['country_code'], country_codes['country_name']))

# Get all country names in order
all_country_names = [country_map.get(cid, f"Unknown_{cid}") for cid in all_country_ids]

# Get all unique HS chapters
all_hs_chapters = sorted(df['hs_chapter'].unique())

# Create output directory for matrices
matrix_output_dir = '../../data/all_trade_matrices'
os.makedirs(matrix_output_dir, exist_ok=True)

def create_trade_matrix(trade_data, hs_chapter=None):
    """
    Create a trade matrix for a specific HS chapter (or all trade if hs_chapter is None)
    
    Returns a DataFrame with:
    - Rows: exporters (with "World" as first row showing total imports)
    - Columns: importers (with "World" as first column showing total exports)
    - Values: trade value in USD
    """
    # Filter by HS chapter if specified
    if hs_chapter is not None:
        filtered_data = trade_data[trade_data['hs_chapter'] == hs_chapter].copy()
    else:
        filtered_data = trade_data.copy()
    
    # Aggregate trade values by exporter-importer pair
    trade_agg = filtered_data.groupby(['i', 'j'])['v'].sum().reset_index()
    
    # Create matrix with all countries (initialize with zeros)
    n_countries = len(all_country_ids)
    matrix = np.zeros((n_countries, n_countries))
    
    # Create mapping from country_id to index
    country_id_to_idx = {cid: idx for idx, cid in enumerate(all_country_ids)}
    
    # Fill in the matrix
    for _, row in trade_agg.iterrows():
        exporter_idx = country_id_to_idx[row['i']]
        importer_idx = country_id_to_idx[row['j']]
        matrix[exporter_idx, importer_idx] = row['v']
    
    # Create DataFrame
    df_matrix = pd.DataFrame(matrix, index=all_country_names, columns=all_country_names)
    
    # Calculate totals
    total_exports = df_matrix.sum(axis=1)  # Sum across columns (total exports per country)
    total_imports = df_matrix.sum(axis=0)  # Sum down rows (total imports per country)
    grand_total = total_exports.sum()  # Total trade
    
    # Add "World" column (total exports)
    df_matrix.insert(0, 'World', total_exports)
    
    # Add "World" row (total imports)
    world_row = pd.Series(['World'] + total_imports.tolist(), index=['World'] + all_country_names)
    world_row['World'] = grand_total
    df_matrix.loc['World'] = world_row
    
    # Move "World" row to the top
    df_matrix = df_matrix.reindex(['World'] + all_country_names)
    
    return df_matrix

# Create matrix for each HS chapter
print(f"\nCreating {len(all_hs_chapters)} HS chapter matrices...")
for hs_chapter in all_hs_chapters:
    matrix = create_trade_matrix(df, hs_chapter=hs_chapter)
    output_path = os.path.join(matrix_output_dir, f'trade_matrix_all_transport_modes_HS{hs_chapter}.csv')
    matrix.to_csv(output_path)
    print(f"✓ Created: trade_matrix_all_transport_modes_HS{hs_chapter}.csv")

# Create TOTAL matrix (all goods)
print(f"\nCreating TOTAL matrix (all goods)...")
total_matrix = create_trade_matrix(df, hs_chapter=None)
total_output_path = os.path.join(matrix_output_dir, 'trade_matrix_all_transport_modes_TOTAL.csv')
total_matrix.to_csv(total_output_path)
print(f"✓ Created: trade_matrix_all_transport_modes_TOTAL.csv")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Conversion factors (general): {len(general_grouped)} HS chapters")
print(f"Trade matrices created: {len(all_hs_chapters) + 1} ({len(all_hs_chapters)} HS chapters + 1 TOTAL)")
print(f"Matrix dimensions: {len(all_country_names) + 1} x {len(all_country_names) + 1} (including World totals)")
print(f"\nAll outputs saved successfully!")

PART 1: CREATING GENERAL CONVERSION FACTOR JSON FILE
✓ General conversion factors exported to: trade_volume_conversion_output/trade_volume_conversion_general.json
  Number of HS chapters: 96

PART 2: CREATING TRADE MATRICES

Creating 96 HS chapter matrices...
✓ Created: trade_matrix_all_transport_modes_HS1.csv
✓ Created: trade_matrix_all_transport_modes_HS2.csv
✓ Created: trade_matrix_all_transport_modes_HS3.csv
✓ Created: trade_matrix_all_transport_modes_HS4.csv
✓ Created: trade_matrix_all_transport_modes_HS5.csv
✓ Created: trade_matrix_all_transport_modes_HS6.csv
✓ Created: trade_matrix_all_transport_modes_HS7.csv
✓ Created: trade_matrix_all_transport_modes_HS8.csv
✓ Created: trade_matrix_all_transport_modes_HS9.csv
✓ Created: trade_matrix_all_transport_modes_HS10.csv
✓ Created: trade_matrix_all_transport_modes_HS11.csv
✓ Created: trade_matrix_all_transport_modes_HS12.csv
✓ Created: trade_matrix_all_transport_modes_HS13.csv
✓ Created: trade_matrix_all_transport_modes_HS14.csv
✓ Creat